<a href="https://colab.research.google.com/github/beaten-by-the-market/dart_disclosure/blob/main/buyback_000390.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 자기주식 '신탁' '처분'은 정말 그렇게 적었을까?

## 파이썬 세팅

* 필요한 패키지 Import

In [1]:
import pandas as pd
import requests
from io import BytesIO, StringIO
from time import sleep
from tqdm import tqdm
from bs4 import BeautifulSoup
import json
import re
import numpy as np

* 시각화를 위한 세팅(코랩)
코랩 환경에서 시각화에 한글이 깨지지 않도록 조치

In [2]:
# 필요한 라이브러리 설치
!pip install adjustText

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from adjustText import adjust_text

# 한글 폰트 설정 (Google Colab 환경에서)
!apt-get update -qq
!apt-get install fonts-nanum* -qq

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=10)

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package fonts-nanum.
(Reading database ... 126209 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-3_all.deb ...
Unpacking fonts-nanum-coding (2.5-3) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-7_all.deb ...
Unpacking fonts-nanum-eco (1.000-7) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20200506-1_all.deb ...
Unpacking fonts-nanum-extra (20200506-1) ...
Setting up fonts-nanum-extra (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Setting up fo

## 사례 : 삼화페인트(000390)

### 정보데이터시스템 20004(자사주취득/처분종목현황) 데이터
- 기간 : 2018.1.1~ 2021.12.31

In [16]:
#거래소 홈페이지에서 불러오기
#화면번호 20004(자사주취득/처분종목현황)

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'locale': 'ko_KR',
    'mktId': 'ALL',
    'tboxisuCd_finder_stkisu0_0': '000390/삼화페인트',
    'isuCd': 'KR7000390005',
    'isuCd2': '000390',
    'codeNmisuCd_finder_stkisu0_0': '삼화페인트',
    'param1isuCd_finder_stkisu0_0': 'ALL',
    'trstkTpCd': 'ALL',
    'trstkAcqstdispTpCd': 'ALL',
    'strtDd': '20180101',
    'endDd': '20211231',
    'share': '1',
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/issue/MDCSTAT20601'
}

headers = {
    'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC03010201',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

otp = requests.post(gen_otp_url, gen_otp_data, headers = headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_content = requests.post(down_url, {'code': otp}, headers = headers)


df_buyback_000390_20004 = pd.read_csv(BytesIO(down_content.content), encoding = 'EUC-KR', dtype={'종목코드':str})

# df_listed DataFrame 사용하여 연도별 유형별 취득수량/처분수량
# '공시일' 칼럼을 날짜형태로 변환
df_buyback_000390_20004['공시일'] = pd.to_datetime(df_buyback_000390_20004['공시일'])

# '공시일' 칼럼에서 연도를 뽑아내기
df_buyback_000390_20004['year'] = df_buyback_000390_20004['공시일'].dt.year

df_buyback_000390_20004

,공시일,종목코드,종목명,시장구분,직접/신탁,취득/처분,취득(처분)예정기간_시작일,취득(처분)예정기간_종료일,신고내역_수량,신고내역_금액,체결내역(누계)_수량,체결내역(누계)_체결수량비율,체결내역(누계)_금액,year
0,2020-02-28,000390,삼화페인트,KOSPI,신탁,처분,2020/02/28,2021/02/28,0,5000000000,469343,0.0,8130409750,2020
1,2018-11-01,000390,삼화페인트,KOSPI,직접,취득,2018/11/02,2019/02/01,948800,5047616000,948800,100.0,6348958490,2018
2,2018-04-10,000390,삼화페인트,KOSPI,직접,취득,2018/04/11,2018/07/10,700000,4620000000,700000,100.0,5677267750,2018


In [17]:
# 숫자 형식 변환 함수
def format_number(value):
    if isinstance(value, (int, float)):
        return f"{value:,.0f}"  # 천 단위 콤마 추가, 소수점 없이 정수 출력
    return value

# 확인해보기
print("행의 개수: " + str(len(df_buyback_000390_20004)))

# 확인해보기
df_show = df_buyback_000390_20004.copy()

# 칼럼에 format_number 적용
df_show[['신고내역_수량','체결내역(누계)_수량','신고내역_금액', '체결내역(누계)_금액']] = df_show[['신고내역_수량','체결내역(누계)_수량','신고내역_금액', '체결내역(누계)_금액']].map(format_number)

# HTML 변환 및 커스텀 태그로 감싸기
html_table = df_show.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
df_show

행의 개수: 3
<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>공시일</th>
      <th>종목코드</th>
      <th>종목명</th>
      <th>시장구분</th>
      <th>직접/신탁</th>
      <th>취득/처분</th>
      <th>취득(처분)예정기간_시작일</th>
      <th>취득(처분)예정기간_종료일</th>
      <th>신고내역_수량</th>
      <th>신고내역_금액</th>
      <th>체결내역(누계)_수량</th>
      <th>체결내역(누계)_체결수량비율</th>
      <th>체결내역(누계)_금액</th>
      <th>year</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>2020-02-28</td>
      <td>000390</td>
      <td>삼화페인트</td>
      <td>KOSPI</td>
      <td>신탁</td>
      <td>처분</td>
      <td>2020/02/28</td>
      <td>2021/02/28</td>
      <td>0</td>
      <td>5,000,000,000</td>
      <td>469,343</td>
      <td>0.0</td>
      <td>8,130,409,750</td>
      <td>2020</td>
    </tr>
    <tr>
      <td>2018-11-01</td>
      <td>000390</td>
      <td>삼화페인트</td>
      <td>KOSPI</td>
      <td>직접</td>
      <td>취득</td>
      <td>2018/11/02</td>
      <td>2019/02/01</td>
      <td>948,8

,공시일,종목코드,종목명,시장구분,직접/신탁,취득/처분,취득(처분)예정기간_시작일,취득(처분)예정기간_종료일,신고내역_수량,신고내역_금액,체결내역(누계)_수량,체결내역(누계)_체결수량비율,체결내역(누계)_금액,year
0,2020-02-28,000390,삼화페인트,KOSPI,신탁,처분,2020/02/28,2021/02/28,0,"5,000,000,000","469,343",0.0,"8,130,409,750",2020
1,2018-11-01,000390,삼화페인트,KOSPI,직접,취득,2018/11/02,2019/02/01,"948,800","5,047,616,000","948,800",100.0,"6,348,958,490",2018
2,2018-04-10,000390,삼화페인트,KOSPI,직접,취득,2018/04/11,2018/07/10,"700,000","4,620,000,000","700,000",100.0,"5,677,267,750",2018


### 정보데이터시스템 20005(자사주취득/처분내역(개별종목)) 데이터

In [33]:
#거래소 홈페이지에서 불러오기
#화면번호 20005(자사주취득/처분내역(개별종목))

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'locale': 'ko_KR',
    'tboxisuCd_finder_stkisu0_1': '000390/삼화페인트',
    'isuCd': 'KR7000390005',
    'isuCd2': '000390',
    'codeNmisuCd_finder_stkisu0_1': '삼화페인트',
    'param1isuCd_finder_stkisu0_1': 'ALL',
    'trstkTpCd': 'ALL',
    'trstkAcqstdispTpCd': 'ALL',
    'strtDd': '20180101',
    'endDd': '20211231',
    'mktId': 'ALL',
    'share': '1',
    'csvxls_isNo': 'true',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/issue/MDCSTAT20701'
}

headers = {
    'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC03010201',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

otp = requests.post(gen_otp_url, gen_otp_data, headers = headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_content = requests.post(down_url, {'code': otp}, headers = headers)


df_buyback_000390 = pd.read_csv(BytesIO(down_content.content), encoding = 'EUC-KR', dtype={'종목코드':str})

# 날짜관련칼럼을 날짜형태로 변환
df_buyback_000390[['신청내역(일별)_신청일','체결내역(일별)_매매일']] = df_buyback_000390[['신청내역(일별)_신청일','체결내역(일별)_매매일']].apply(pd.to_datetime)

In [35]:
# 확인해보기
print("행의 개수: " + str(len(df_buyback_000390)))

# 상위 10개만 확인해보기
print("상위 10개만 확인")
df_show = df_buyback_000390.copy().head(10)

# 칼럼에 format_number 적용
df_show[['신청내역(일별)_신청가능수량', '신청내역(일별)_신청수량(A)', '체결내역(일별)_체결수량(B)']] = df_show[['신청내역(일별)_신청가능수량', '신청내역(일별)_신청수량(A)', '체결내역(일별)_체결수량(B)']].map(format_number)

# HTML 변환 및 커스텀 태그로 감싸기
html_table = df_show.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
df_show

행의 개수: 122
상위 10개만 확인
<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>번호</th>
      <th>종목코드</th>
      <th>종목명</th>
      <th>시장구분</th>
      <th>직접/신탁</th>
      <th>취득/처분</th>
      <th>신청내역(일별)_신청일</th>
      <th>신청내역(일별)_신청가능수량</th>
      <th>신청내역(일별)_신청수량(A)</th>
      <th>체결내역(일별)_매매일</th>
      <th>체결내역(일별)_체결수량(B)</th>
      <th>체결내역(일별)_체결율(B/A)</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1</td>
      <td>000390</td>
      <td>삼화페인트</td>
      <td>KOSPI</td>
      <td>신탁</td>
      <td>처분</td>
      <td>2020-12-21</td>
      <td>0</td>
      <td>69,343</td>
      <td>2020-12-22</td>
      <td>69,343</td>
      <td>100.00</td>
    </tr>
    <tr>
      <td>2</td>
      <td>000390</td>
      <td>삼화페인트</td>
      <td>KOSPI</td>
      <td>신탁</td>
      <td>처분</td>
      <td>2020-12-18</td>
      <td>0</td>
      <td>200,000</td>
      <td>2020-12-21</td>
      <td>200,000</td>
      <td>100.00</td>
    </tr>
    <tr

,번호,종목코드,종목명,시장구분,직접/신탁,취득/처분,신청내역(일별)_신청일,신청내역(일별)_신청가능수량,신청내역(일별)_신청수량(A),체결내역(일별)_매매일,체결내역(일별)_체결수량(B),체결내역(일별)_체결율(B/A)
0,1,000390,삼화페인트,KOSPI,신탁,처분,2020-12-21,0,"69,343",2020-12-22,"69,343",100.00
1,2,000390,삼화페인트,KOSPI,신탁,처분,2020-12-18,0,"200,000",2020-12-21,"200,000",100.00
2,3,000390,삼화페인트,KOSPI,신탁,처분,2020-12-17,0,"200,000",2020-12-18,"200,000",100.00
3,4,000390,삼화페인트,KOSPI,신탁,취득,2020-04-16,0,"5,000",2020-04-17,0,0.00
4,5,000390,삼화페인트,KOSPI,신탁,취득,2020-04-14,0,"5,000",2020-04-16,256,5.12
5,6,000390,삼화페인트,KOSPI,신탁,취득,2020-04-13,0,"5,000",2020-04-14,"3,075",61.50
6,7,000390,삼화페인트,KOSPI,신탁,취득,2020-04-10,0,"5,000",2020-04-13,"5,000",100.00
7,8,000390,삼화페인트,KOSPI,신탁,취득,2020-04-09,0,"10,000",2020-04-10,"8,000",80.00
8,9,000390,삼화페인트,KOSPI,신탁,취득,2020-04-08,0,"10,000",2020-04-09,"2,964",29.64
9,10,000390,삼화페인트,KOSPI,신탁,취득,2020-04-07,0,"10,000",2020-04-08,"3,646",36.46


In [36]:
# '체결내역(일별)_매매일' 칼럼에서 연도를 뽑아내기
df_buyback_000390['year'] = df_buyback_000390['체결내역(일별)_매매일'].dt.year

# 기준을 정하여 그룹화
grouped_000390 = df_buyback_000390.groupby(['시장구분','year', '직접/신탁', '취득/처분']).agg({
    '체결내역(일별)_체결수량(B)' : 'sum'
}).reset_index()

# 확인해보기
df_show = grouped_000390.copy()

# df_show에서 시장구분 칼럼 삭제하기
df_show = df_show.drop('시장구분', axis=1)

# 칼럼에 format_number 적용
df_show[['체결내역(일별)_체결수량(B)']] = df_show[['체결내역(일별)_체결수량(B)']].map(format_number)

# HTML 변환 및 커스텀 태그로 감싸기
html_table = df_show.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
df_show

<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>year</th>
      <th>직접/신탁</th>
      <th>취득/처분</th>
      <th>체결내역(일별)_체결수량(B)</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>2018</td>
      <td>직접</td>
      <td>취득</td>
      <td>1,436,770</td>
    </tr>
    <tr>
      <td>2019</td>
      <td>직접</td>
      <td>취득</td>
      <td>212,030</td>
    </tr>
    <tr>
      <td>2020</td>
      <td>신탁</td>
      <td>처분</td>
      <td>469,343</td>
    </tr>
    <tr>
      <td>2020</td>
      <td>신탁</td>
      <td>취득</td>
      <td>469,343</td>
    </tr>
  </tbody>
</table>


,year,직접/신탁,취득/처분,체결내역(일별)_체결수량(B)
0,2018,직접,취득,"1,436,770"
1,2019,직접,취득,"212,030"
2,2020,신탁,처분,"469,343"
3,2020,신탁,취득,"469,343"


## 사례 : 만호제강(001080)

### 정보데이터시스템 20004(자사주취득/처분종목현황) 데이터
- 기간 : 2017.1.1~ 2024.12.31

In [31]:
#거래소 홈페이지에서 불러오기
#화면번호 20004(자사주취득/처분종목현황)

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'locale': 'ko_KR',
    'mktId': 'ALL',
    'tboxisuCd_finder_stkisu0_0': '001080/만호제강',
    'isuCd': 'KR7001080001',
    'isuCd2': '001080',
    'codeNmisuCd_finder_stkisu0_0': '만호제강',
    'param1isuCd_finder_stkisu0_0': 'ALL',
    'trstkTpCd': 'ALL',
    'trstkAcqstdispTpCd': 'ALL',
    'strtDd': '20170101',
    'endDd': '20241231',
    'share': '1',
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/issue/MDCSTAT20601'
}

headers = {
    'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC03010201',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

otp = requests.post(gen_otp_url, gen_otp_data, headers = headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_content = requests.post(down_url, {'code': otp}, headers = headers)


df_buyback_001080_20004 = pd.read_csv(BytesIO(down_content.content), encoding = 'EUC-KR', dtype={'종목코드':str})

# df_listed DataFrame 사용하여 연도별 유형별 취득수량/처분수량
# '공시일' 칼럼을 날짜형태로 변환
df_buyback_001080_20004['공시일'] = pd.to_datetime(df_buyback_001080_20004['공시일'])

# '공시일' 칼럼에서 연도를 뽑아내기
df_buyback_001080_20004['year'] = df_buyback_001080_20004['공시일'].dt.year

In [32]:
# 숫자 형식 변환 함수
def format_number(value):
    if isinstance(value, (int, float)):
        return f"{value:,.0f}"  # 천 단위 콤마 추가, 소수점 없이 정수 출력
    return value

# 확인해보기
print("행의 개수: " + str(len(df_buyback_001080_20004)))

# 확인해보기
df_show = df_buyback_001080_20004.copy()

# 칼럼에 format_number 적용
df_show[['신고내역_수량','체결내역(누계)_수량','신고내역_금액', '체결내역(누계)_금액']] = df_show[['신고내역_수량','체결내역(누계)_수량','신고내역_금액', '체결내역(누계)_금액']].map(format_number)

# HTML 변환 및 커스텀 태그로 감싸기
html_table = df_show.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
df_show

행의 개수: 5
<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>공시일</th>
      <th>종목코드</th>
      <th>종목명</th>
      <th>시장구분</th>
      <th>직접/신탁</th>
      <th>취득/처분</th>
      <th>취득(처분)예정기간_시작일</th>
      <th>취득(처분)예정기간_종료일</th>
      <th>신고내역_수량</th>
      <th>신고내역_금액</th>
      <th>체결내역(누계)_수량</th>
      <th>체결내역(누계)_체결수량비율</th>
      <th>체결내역(누계)_금액</th>
      <th>year</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>2023-06-07</td>
      <td>001080</td>
      <td>만호제강</td>
      <td>KOSPI</td>
      <td>직접</td>
      <td>처분</td>
      <td>2023/06/07</td>
      <td>2023/06/16</td>
      <td>289,000</td>
      <td>19,016,200,000</td>
      <td>150,000</td>
      <td>51.90</td>
      <td>9,390,000,000</td>
      <td>2023</td>
    </tr>
    <tr>
      <td>2023-05-11</td>
      <td>001080</td>
      <td>만호제강</td>
      <td>KOSPI</td>
      <td>직접</td>
      <td>처분</td>
      <td>2023/05/11</td>
      <td>2023/05/18</td>
      <t

,공시일,종목코드,종목명,시장구분,직접/신탁,취득/처분,취득(처분)예정기간_시작일,취득(처분)예정기간_종료일,신고내역_수량,신고내역_금액,체결내역(누계)_수량,체결내역(누계)_체결수량비율,체결내역(누계)_금액,year
0,2023-06-07,001080,만호제강,KOSPI,직접,처분,2023/06/07,2023/06/16,"289,000","19,016,200,000","150,000",51.90,"9,390,000,000",2023
1,2023-05-11,001080,만호제강,KOSPI,직접,처분,2023/05/11,2023/05/18,"200,000","12,880,000,000","200,000",100.00,"12,580,000,000",2023
2,2021-11-18,001080,만호제강,KOSPI,직접,취득,2021/11/19,2022/02/18,"150,000","3,165,000,000","150,000",100.00,"3,261,928,550",2021
3,2020-03-11,001080,만호제강,KOSPI,직접,취득,2020/03/12,2020/06/11,"100,000","1,350,000,000","59,625",59.63,"687,746,350",2020
4,2018-03-22,001080,만호제강,KOSPI,직접,취득,2018/03/23,2018/06/22,"100,000","1,765,000,000","41,796",41.80,"742,848,400",2018


### 정보데이터시스템 20005(자사주취득/처분내역(개별종목)) 데이터

In [48]:
#거래소 홈페이지에서 불러오기
#화면번호 20005(자사주취득/처분내역(개별종목))

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'locale': 'ko_KR',
    'tboxisuCd_finder_stkisu0_1': '001080/만호제강',
    'isuCd': 'KR7001080001',
    'isuCd2': '001080',
    'codeNmisuCd_finder_stkisu0_1': '만호제강',
    'param1isuCd_finder_stkisu0_1': 'ALL',
    'trstkTpCd': 'ALL',
    'trstkAcqstdispTpCd': 'ALL',
    'strtDd': '20170101',
    'endDd': '20241231',
    'mktId': 'ALL',
    'share': '1',
    'csvxls_isNo': 'true',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/issue/MDCSTAT20701'
}

headers = {
    'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC03010201',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

otp = requests.post(gen_otp_url, gen_otp_data, headers = headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_content = requests.post(down_url, {'code': otp}, headers = headers)


df_buyback_001080 = pd.read_csv(BytesIO(down_content.content), encoding = 'EUC-KR', dtype={'종목코드':str})

# 날짜관련칼럼을 날짜형태로 변환
df_buyback_001080[['신청내역(일별)_신청일','체결내역(일별)_매매일']] = df_buyback_001080[['신청내역(일별)_신청일','체결내역(일별)_매매일']].apply(pd.to_datetime)

In [49]:
# 확인해보기
print("행의 개수: " + str(len(df_buyback_001080)))

# 상위 10개만 확인해보기
print("상위 10개만 확인")
df_show = df_buyback_001080.copy().head()

# 칼럼에 format_number 적용
df_show[['신청내역(일별)_신청가능수량', '신청내역(일별)_신청수량(A)', '체결내역(일별)_체결수량(B)']] = df_show[['신청내역(일별)_신청가능수량', '신청내역(일별)_신청수량(A)', '체결내역(일별)_체결수량(B)']].map(format_number)

# HTML 변환 및 커스텀 태그로 감싸기
html_table = df_show.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
df_show

행의 개수: 153
상위 10개만 확인
<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>번호</th>
      <th>종목코드</th>
      <th>종목명</th>
      <th>시장구분</th>
      <th>직접/신탁</th>
      <th>취득/처분</th>
      <th>신청내역(일별)_신청일</th>
      <th>신청내역(일별)_신청가능수량</th>
      <th>신청내역(일별)_신청수량(A)</th>
      <th>체결내역(일별)_매매일</th>
      <th>체결내역(일별)_체결수량(B)</th>
      <th>체결내역(일별)_체결율(B/A)</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1</td>
      <td>001080</td>
      <td>만호제강</td>
      <td>KOSPI</td>
      <td>신탁</td>
      <td>처분</td>
      <td>2024-10-30</td>
      <td>0</td>
      <td>362</td>
      <td>2024-10-31</td>
      <td>362</td>
      <td>100.0</td>
    </tr>
    <tr>
      <td>2</td>
      <td>001080</td>
      <td>만호제강</td>
      <td>KOSPI</td>
      <td>신탁</td>
      <td>처분</td>
      <td>2024-10-29</td>
      <td>0</td>
      <td>372</td>
      <td>2024-10-30</td>
      <td>372</td>
      <td>100.0</td>
    </tr>
    <tr>
      <td>3</td>

,번호,종목코드,종목명,시장구분,직접/신탁,취득/처분,신청내역(일별)_신청일,신청내역(일별)_신청가능수량,신청내역(일별)_신청수량(A),체결내역(일별)_매매일,체결내역(일별)_체결수량(B),체결내역(일별)_체결율(B/A)
0,1,001080,만호제강,KOSPI,신탁,처분,2024-10-30,0,362,2024-10-31,362,100.0
1,2,001080,만호제강,KOSPI,신탁,처분,2024-10-29,0,372,2024-10-30,372,100.0
2,3,001080,만호제강,KOSPI,신탁,처분,2024-10-28,0,372,2024-10-29,372,100.0
3,4,001080,만호제강,KOSPI,신탁,처분,2024-10-25,0,372,2024-10-28,372,100.0
4,5,001080,만호제강,KOSPI,신탁,처분,2024-10-24,0,372,2024-10-25,372,100.0


In [50]:
# '체결내역(일별)_매매일' 칼럼에서 연도를 뽑아내기
df_buyback_001080['year'] = df_buyback_001080['체결내역(일별)_매매일'].dt.year

# 기준을 정하여 그룹화
grouped_000390 = df_buyback_001080.groupby(['시장구분','year', '직접/신탁', '취득/처분']).agg({
    '체결내역(일별)_체결수량(B)' : 'sum'
}).reset_index()

# 확인해보기
df_show = grouped_000390.copy()

# 칼럼에 format_number 적용
df_show[['체결내역(일별)_체결수량(B)']] = df_show[['체결내역(일별)_체결수량(B)']].map(format_number)

# df_show에서 시장구분 칼럼 삭제하기
df_show = df_show.drop('시장구분', axis=1)

# HTML 변환 및 커스텀 태그로 감싸기
html_table = df_show.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
df_show

<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>year</th>
      <th>직접/신탁</th>
      <th>취득/처분</th>
      <th>체결내역(일별)_체결수량(B)</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>2017</td>
      <td>신탁</td>
      <td>처분</td>
      <td>6,581</td>
    </tr>
    <tr>
      <td>2018</td>
      <td>신탁</td>
      <td>처분</td>
      <td>1,091</td>
    </tr>
    <tr>
      <td>2018</td>
      <td>직접</td>
      <td>취득</td>
      <td>41,796</td>
    </tr>
    <tr>
      <td>2019</td>
      <td>신탁</td>
      <td>처분</td>
      <td>1,327</td>
    </tr>
    <tr>
      <td>2020</td>
      <td>신탁</td>
      <td>처분</td>
      <td>934</td>
    </tr>
    <tr>
      <td>2020</td>
      <td>직접</td>
      <td>취득</td>
      <td>59,625</td>
    </tr>
    <tr>
      <td>2021</td>
      <td>신탁</td>
      <td>처분</td>
      <td>1,109</td>
    </tr>
    <tr>
      <td>2021</td>
      <td>직접</td>
      <td>취득</td>
      <td>146,940</td>
    </tr>
    <tr>
   

,year,직접/신탁,취득/처분,체결내역(일별)_체결수량(B)
0,2017,신탁,처분,"6,581"
1,2018,신탁,처분,"1,091"
2,2018,직접,취득,"41,796"
3,2019,신탁,처분,"1,327"
4,2020,신탁,처분,934
5,2020,직접,취득,"59,625"
6,2021,신탁,처분,"1,109"
7,2021,직접,취득,"146,940"
8,2022,신탁,처분,832
9,2022,직접,취득,"3,060"


In [56]:
# 비교를 위한 데이터프레임 생성
# 20005화면에서 직접, 신탁을 구분하기
df_show_direct = df_show[df_show['직접/신탁']=='직접']
df_show_sh = df_show[df_show['직접/신탁']=='신탁']

# 20004화면에서 필요없는 칼럼 제거하고 포맷 바꾸기
df_buyback_001080_20004_anal = df_buyback_001080_20004[['공시일', 'year','직접/신탁', '취득/처분','체결내역(누계)_수량', '취득(처분)예정기간_시작일','취득(처분)예정기간_종료일']]

# 칼럼에 format_number 적용
df_buyback_001080_20004_anal = df_buyback_001080_20004_anal.copy().sort_values(by='year')
df_buyback_001080_20004_anal[['체결내역(누계)_수량']] = df_buyback_001080_20004_anal[['체결내역(누계)_수량']].map(format_number)

In [61]:
# HTML 변환 및 커스텀 태그로 감싸기
html_table = df_buyback_001080_20004_anal.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
df_buyback_001080_20004_anal

<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>공시일</th>
      <th>year</th>
      <th>직접/신탁</th>
      <th>취득/처분</th>
      <th>체결내역(누계)_수량</th>
      <th>취득(처분)예정기간_시작일</th>
      <th>취득(처분)예정기간_종료일</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>2018-03-22</td>
      <td>2018</td>
      <td>직접</td>
      <td>취득</td>
      <td>41,796</td>
      <td>2018/03/23</td>
      <td>2018/06/22</td>
    </tr>
    <tr>
      <td>2020-03-11</td>
      <td>2020</td>
      <td>직접</td>
      <td>취득</td>
      <td>59,625</td>
      <td>2020/03/12</td>
      <td>2020/06/11</td>
    </tr>
    <tr>
      <td>2021-11-18</td>
      <td>2021</td>
      <td>직접</td>
      <td>취득</td>
      <td>150,000</td>
      <td>2021/11/19</td>
      <td>2022/02/18</td>
    </tr>
    <tr>
      <td>2023-05-11</td>
      <td>2023</td>
      <td>직접</td>
      <td>처분</td>
      <td>200,000</td>
      <td>2023/05/11</td>
      <td>2023/05/18</td>
    </tr>
    <t

,공시일,year,직접/신탁,취득/처분,체결내역(누계)_수량,취득(처분)예정기간_시작일,취득(처분)예정기간_종료일
4,2018-03-22,2018,직접,취득,"41,796",2018/03/23,2018/06/22
3,2020-03-11,2020,직접,취득,"59,625",2020/03/12,2020/06/11
2,2021-11-18,2021,직접,취득,"150,000",2021/11/19,2022/02/18
1,2023-05-11,2023,직접,처분,"200,000",2023/05/11,2023/05/18
0,2023-06-07,2023,직접,처분,"150,000",2023/06/07,2023/06/16


In [62]:
# HTML 변환 및 커스텀 태그로 감싸기
html_table = df_show_direct.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
df_show_direct

<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>year</th>
      <th>직접/신탁</th>
      <th>취득/처분</th>
      <th>체결내역(일별)_체결수량(B)</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>2018</td>
      <td>직접</td>
      <td>취득</td>
      <td>41,796</td>
    </tr>
    <tr>
      <td>2020</td>
      <td>직접</td>
      <td>취득</td>
      <td>59,625</td>
    </tr>
    <tr>
      <td>2021</td>
      <td>직접</td>
      <td>취득</td>
      <td>146,940</td>
    </tr>
    <tr>
      <td>2022</td>
      <td>직접</td>
      <td>취득</td>
      <td>3,060</td>
    </tr>
    <tr>
      <td>2023</td>
      <td>직접</td>
      <td>처분</td>
      <td>350,000</td>
    </tr>
  </tbody>
</table>


,year,직접/신탁,취득/처분,체결내역(일별)_체결수량(B)
2,2018,직접,취득,"41,796"
5,2020,직접,취득,"59,625"
7,2021,직접,취득,"146,940"
9,2022,직접,취득,"3,060"
11,2023,직접,처분,"350,000"


In [63]:
# HTML 변환 및 커스텀 태그로 감싸기
html_table = df_show_sh.to_html(index=False, classes="dataframe", escape=False)

# 테이블을 <div> 태그로 감싸기
wrapped_html = f"""
<div class="table-container">
  {html_table}
</div>
"""
print(html_table)
df_show_sh

<table border="1" class="dataframe dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>year</th>
      <th>직접/신탁</th>
      <th>취득/처분</th>
      <th>체결내역(일별)_체결수량(B)</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>2017</td>
      <td>신탁</td>
      <td>처분</td>
      <td>6,581</td>
    </tr>
    <tr>
      <td>2018</td>
      <td>신탁</td>
      <td>처분</td>
      <td>1,091</td>
    </tr>
    <tr>
      <td>2019</td>
      <td>신탁</td>
      <td>처분</td>
      <td>1,327</td>
    </tr>
    <tr>
      <td>2020</td>
      <td>신탁</td>
      <td>처분</td>
      <td>934</td>
    </tr>
    <tr>
      <td>2021</td>
      <td>신탁</td>
      <td>처분</td>
      <td>1,109</td>
    </tr>
    <tr>
      <td>2022</td>
      <td>신탁</td>
      <td>처분</td>
      <td>832</td>
    </tr>
    <tr>
      <td>2023</td>
      <td>신탁</td>
      <td>처분</td>
      <td>50,720</td>
    </tr>
    <tr>
      <td>2024</td>
      <td>신탁</td>
      <td>처분</td>
      <td>46,594</td>
    </tr>
  </tbody>
</tab

,year,직접/신탁,취득/처분,체결내역(일별)_체결수량(B)
0,2017,신탁,처분,"6,581"
1,2018,신탁,처분,"1,091"
3,2019,신탁,처분,"1,327"
4,2020,신탁,처분,934
6,2021,신탁,처분,"1,109"
8,2022,신탁,처분,832
10,2023,신탁,처분,"50,720"
12,2024,신탁,처분,"46,594"


## 20005 화면으로 전종목 확인해보기

In [ ]:
#거래소 홈페이지에서 불러오기
# 조회연도 입력하기(2015~2024)
st_y = '2015'
end_y = '2024'
#화면번호 20005(자사주취득/처분내역(개별종목))

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'locale': 'ko_KR',
    'tboxisuCd_finder_stkisu0_2': '전체',
    'isuCd': 'ALL',
    'isuCd2': 'ALL',
    'codeNmisuCd_finder_stkisu0_2': '',
    'param1isuCd_finder_stkisu0_2': 'ALL',
    'trstkTpCd': 'ALL',
    'trstkAcqstdispTpCd': 'ALL',
    'strtDd': f'{st_y}0101',
    'endDd': f'{end_y}1224',
    'mktId': 'ALL',
    'share': '1',
    'csvxls_isNo': 'true',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/issue/MDCSTAT20701'
}

headers = {
    'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC03010201',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

otp = requests.post(gen_otp_url, gen_otp_data, headers = headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_content = requests.post(down_url, {'code': otp}, headers = headers)

df_buyback = pd.read_csv(BytesIO(down_content.content), encoding = 'EUC-KR', dtype=str)

In [ ]:
# 저장
df_buyback.to_csv('/content/drive/My Drive/beatenbythemarket/df_buyback.csv', index=False, encoding='utf-8-sig')

# 모든 데이터를 문자열 형태로 불러오기
# 파일경로 지정
file_path = '/content/drive/My Drive/beatenbythemarket/df_buyback.csv'

# 불러오기
df_price_info = pd.read_csv(file_path, dtype=str, encoding='utf-8-sig')
# 확인
df_buyback

,번호,종목코드,종목명,시장구분,직접/신탁,취득/처분,신청내역(일별)_신청일,신청내역(일별)_신청가능수량,신청내역(일별)_신청수량(A),체결내역(일별)_매매일,체결내역(일별)_체결수량(B),체결내역(일별)_체결율(B/A)
0,1,001460,BYC,KOSPI,신탁,취득,2024/12/24,0,1000,2024/12/26,1000,100.00
1,2,001465,BYC우,KOSPI,신탁,취득,2024/12/24,0,1000,2024/12/26,1000,100.00
2,3,000590,CS홀딩스,KOSPI,신탁,취득,2024/12/24,0,200,2024/12/26,200,100.00
3,4,241520,DSC인베스트먼트,KOSDAQ,신탁,취득,2024/12/24,0,19000,2024/12/26,18917,99.56
4,5,001060,JW중외제약,KOSPI,직접,취득,2024/12/24,199456,5550,2024/12/26,3330,60.00
...,...,...,...,...,...,...,...,...,...,...,...,...
146829,146830,090850,현대이지웰,KOSDAQ,신탁,취득,2015/01/02,0,20000,2015/01/05,0,0.00
146830,146831,005380,현대차,KOSPI,직접,취득,2015/01/02,1356897,110000,2015/01/05,96551,87.77
146831,146832,005387,현대차2우B,KOSPI,직접,취득,2015/01/02,286196,20000,2015/01/05,20000,100.00
146832,146833,005389,현대차3우B,KOSPI,직접,취득,2015/01/02,17301,1200,2015/01/05,1135,94.58
